Coding a Neural Network from Scratch

Activation Function

In [7]:
import numpy as np

def sigmoid(x):
    # Our activation function: f(x) = 1 / (1 + e^(-x))
    return 1/ (1 + np.exp(-x))

Creating a Neuron

In [8]:
class Neuron:

    def __init__(self, weight, bias):
        self.weight = weight
        self.bias = bias
    
    def feed_forward(self, inputs):
        # Weight inputs, add bias, then use the activation function
        total = np.dot(self.weight, inputs) + self.bias
        return sigmoid(total)

Implementing a Neuron

In [9]:
weights = np.array([0,1]) # w1 = 0, w2 = 1
bias = 4 # b = 4

n = Neuron(weights, bias)

x = np.array([2,3]) # x1 = 2, x2 = 3

print(n.feed_forward(x)) # 0.9990889488055994

0.9990889488055994


Combining Neurons into a Neural Network

In [10]:
class Neural_Network:
    '''
    A Neural Network with:
        1. 2 Inputs
        2. A hidden layer with 2 neurons (h1, h2)
        3. An output layer with 1 neuron (o1)
    Each Neuron has the same weight and bias:
        w = [0,1]
        b = 0
    '''

    def __init__(self):
        weights = np.array([0,1])
        bias = 0

        # Neurons
        self.h1 = Neuron(weights, bias)
        self.h2 = Neuron(weights, bias)
        self.o1 = Neuron(weights, bias)
    
    def feed_forward(self, x):
        out_h1 = self.h1.feed_forward(x)
        out_h2 = self.h2.feed_forward(x)

        # The inputs for o1 are the outputs of h1 and h2
        out_o1 = self.o1.feed_forward(np.array([out_h1, out_h2]))

        return out_o1

In [11]:
network = Neural_Network()
x = np.array([2,3])
print(network.feed_forward(x)) # 0.7216325609518421

0.7216325609518421
